In [1]:
# -*- coding: utf-8 -*-
"""
MetaTrader5 - other important api calls

@author: Mayank
"""

import MetaTrader5 as mt5
import pandas as pd
import os

os.chdir(r"C:\DS Learning\Algorithmic Trading\Automated_Trading\Quant\MT5 Trading Terminal") #path where login credentials and server details
key = open("keys.txt","r").read().split()
path = r"C:\Program Files\MetaTrader 5\terminal64.exe"


# establish MetaTrader 5 connection to a specified trading account
if mt5.initialize(path=path,login=int(key[0]), password=key[1], server=key[2]):
    print("connection established")

def get_position_df():
    positions = mt5.positions_get()
    if len(positions) > 0:
        pos_df = pd.DataFrame(list(positions),columns=positions[0]._asdict().keys())
        pos_df.time = pd.to_datetime(pos_df.time, unit="s")
        pos_df.drop(['time_update', 'time_msc', 'time_update_msc', 'external_id'], axis=1, inplace=True)
    else:
        pos_df = pd.DataFrame()
        
    return pos_df

def get_orders_df():
    orders = mt5.orders_get()
    if len(orders) > 0:
        ord_df = pd.DataFrame(list(orders),columns=orders[0]._asdict().keys())
        ord_df.time_setup = pd.to_datetime(ord_df.time_setup , unit="s")
        #ord_df.drop(['time_update_msc'], axis=1, inplace=True)
    else:
        ord_df = pd.DataFrame()
        
    return ord_df

connection established


In [2]:
import MetaTrader5 as mt5
import pandas as pd
pd.set_option('display.max_columns', 500) # number of columns to be displayed
pd.set_option('display.width', 1500)      # max table width to display
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)
print()
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288



In [4]:
# get open positions on USDCHF
positions=mt5.positions_get()
if positions==None:
    print("No positions on USDCHF, error code={}".format(mt5.last_error()))
elif len(positions)>0:
    print("Total positions on USDCHF =",len(positions))
    # display all open positions
    for position in positions:
        print(position)

Total positions on USDCHF = 2
TradePosition(ticket=51293458046, time=1717610188, time_msc=1717610188554, time_update=1717610188, time_update_msc=1717610188554, type=0, magic=0, identifier=51293458046, reason=3, volume=0.02, price_open=1.27563, sl=0.0, tp=0.0, price_current=1.27747, swap=0.0, profit=3.68, symbol='GBPUSD', comment='', external_id='')
TradePosition(ticket=51293503026, time=1717610457, time_msc=1717610457519, time_update=1717610457, time_update_msc=1717610457519, type=0, magic=0, identifier=51293503026, reason=0, volume=0.01, price_open=1.08667, sl=0.0, tp=0.0, price_current=1.08685, swap=0.0, profit=0.18, symbol='EURUSD', comment='', external_id='')


### check the MT5 Stratergy

In [14]:
import MetaTrader5 as mt5
import os
import numpy as np
import pandas as pd
import datetime as dt
from stocktrends import Renko
import time
import copy

os.chdir(r"C:\DS Learning\Algorithmic Trading\Automated_Trading\Quant\MT5 Trading Terminal") #path where login credentials and server details
key = open("keys.txt","r").read().split()
path = r"C:\Program Files\MetaTrader 5\terminal64.exe"


# establish MetaTrader 5 connection to a specified trading account
if mt5.initialize(path=path, login=int(key[0]), password=key[1], server=key[2]):
    print("connection established")

#defining strategy parameters
pairs = ['EURUSD','GBPUSD','USDCHF','AUDUSD','USDCAD'] #currency pairs to be included in the strategy
pos_size = 0.5 #max capital allocated/position size for any currency pair. in MT5 the size is in unit of 10^5


def MACD(DF,a,b,c):
    """function to calculate MACD
       typical values a = 12; b =26, c =9"""
    df = DF.copy()
    df["MA_Fast"]=df["Close"].ewm(span=a,min_periods=a).mean()
    df["MA_Slow"]=df["Close"].ewm(span=b,min_periods=b).mean()
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"]
    df["Signal"]=df["MACD"].ewm(span=c,min_periods=c).mean()
    df.dropna(inplace=True)
    return (df["MACD"],df["Signal"])

def ATR(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2

def renko_DF(DF):
    "function to convert ohlc data into renko bricks"
    print("Columns before resetting index:", DF.columns)  # Debugging statement
    print("Length of columns before resetting index:", len(DF.columns))  # Debugging statement
    df = DF.reset_index(drop=True)  # Drop existing index before resetting
    print("Columns after resetting index:", df.columns)  # Debugging statement
    print("Length of columns after resetting index:", len(df.columns))  # Debugging statement
    df.columns = ["Date", "Open", "Close", "High", "Low", "Volume"]  # Corrected column names
    renko_df = Renko(df)
    renko_df.brick_size = round(ATR(DF,120)["ATR"].iloc[-1],4)
    renko_df = renko_df.get_ohlc_data()
    renko_df["bar_num"] = np.where(renko_df["uptrend"]==True,1,np.where(renko_df["uptrend"]==False,-1,0))
    for i in range(1,len(renko_df["bar_num"])):
        if renko_df["bar_num"].iloc[i]>0 and renko_df["bar_num"].iloc[i-1]>0:
            renko_df.loc[i, "bar_num"] += renko_df.loc[i-1, "bar_num"]
        elif renko_df["bar_num"].iloc[i]<0 and renko_df["bar_num"].iloc[i-1]<0:
            renko_df.loc[i, "bar_num"] += renko_df.loc[i-1, "bar_num"]
    renko_df.drop_duplicates(subset="Date",keep="last", inplace=True)
    return renko_df



def renko_merge(DF):
    "function to merging renko df with original ohlc df"
    df = copy.deepcopy(DF)
    df["Date"] = df.index
    renko = renko_DF(df)
    renko.columns = ["Date","open","high","low","close","uptrend","bar_num"]
    merged_df = df.merge(renko.loc[:,["Date","bar_num"]],how="outer",on="Date")
    merged_df["bar_num"].fillna(method='ffill',inplace=True)
    merged_df["macd"]= MACD(merged_df,12,26,9)[0]
    merged_df["macd_sig"]= MACD(merged_df,12,26,9)[1]
    return merged_df

def get_position_df():
    positions = mt5.positions_get()
    if len(positions) > 0:
        pos_df = pd.DataFrame(list(positions),columns=positions[0]._asdict().keys())
        pos_df.time = pd.to_datetime(pos_df.time, unit="s")
        pos_df.drop(['time_update', 'time_msc', 'time_update_msc', 'external_id'], axis=1, inplace=True)
        pos_df.type = np.where(pos_df.type==0,1,-1) #to distinguish between long and short positions
    else:
        pos_df = pd.DataFrame()
        
    return pos_df

def get_5m_candles(currency,lookback=10,bars=250):
    data = mt5.copy_rates_from(currency, mt5.TIMEFRAME_M5, dt.datetime.now() - dt.timedelta(10), 250)   
    data_df = pd.DataFrame(data) 
    data_df['time'] = pd.to_datetime(data_df['time'], unit='s')  # Convert 'time' column to datetime
    data_df.set_index("time", inplace=True)
    data_df.rename(columns={"open":"Open","high":"High","low":"Low","close":"Close","volume":"Volume"}, inplace=True)
    data_df.index.name = "Date"  # Rename index to 'Date'
    return data_df

def place_market_order(symbol,vol,buy_sell):
    if buy_sell.capitalize()[0] == "B":
        direction = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    else:
        direction = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": vol,
        "type": direction,
        "price": price,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    
    result = mt5.order_send(request)
    return result

def trade_signal(MERGED_DF,l_s):
    "function to generate signal"
    signal = ""
    df = copy.deepcopy(MERGED_DF)
    if l_s == "":
        if df["bar_num"].tolist()[-1]>=2 and df["macd"].tolist()[-1]>df["macd_sig"].tolist()[-1]:
            signal = "Buy"
        elif df["bar_num"].tolist()[-1]<=-2 and df["macd"].tolist()[-1]<df["macd_sig"].tolist()[-1]:
            signal = "Sell"
    elif l_s == "long":
        if df["bar_num"].tolist()[-1]<=-2 and df["macd"].tolist()[-1]<df["macd_sig"].tolist()[-1]:
            signal = "Close"
        elif df["bar_num"].tolist()[-1]>=2 and df["macd"].tolist()[-1]>df["macd_sig"].tolist()[-1]:
            signal = "Hold"
    elif l_s == "short":
        if df["bar_num"].tolist()[-1]>=2 and df["macd"].tolist()[-1]>df["macd_sig"].tolist()[-1]:
            signal = "Close"
        elif df["bar_num"].tolist()[-1]<=-2 and df["macd"].tolist()[-1]<df["macd_sig"].tolist()[-1]:
            signal = "Hold"
    return signal

def main():
    while True:
        signals = {}
        pos_df = get_position_df()
        for currency in pairs:
            data = get_5m_candles(currency)
            merged_df = renko_merge(data)
            signal = trade_signal(merged_df,pos_df)
            signals[currency] = signal
            time.sleep(5)
        print(signals)
        # logic for executing trades based on signals
        for currency, signal in signals.items():
            if signal == "Buy":
                place_market_order(currency, pos_size, "buy")
            elif signal == "Sell":
                place_market_order(currency, pos_size, "sell")
            elif signal == "Close":
                # Close existing position
                if len(pos_df) > 0:
                    for i, row in pos_df.iterrows():
                        if row['symbol'] == currency:
                            if row['type'] == 1:  # Close long position
                                place_market_order(currency, row['volume'], "sell")
                            elif row['type'] == -1:  # Close short position
                                place_market_order(currency, row['volume'], "buy")
            time.sleep(5)

if __name__ == "__main__":
    main()


connection established
Columns before resetting index: Index(['Open', 'High', 'Low', 'Close', 'tick_volume', 'spread', 'real_volume', 'Date'], dtype='object')
Length of columns before resetting index: 8
Columns after resetting index: Index(['Open', 'High', 'Low', 'Close', 'tick_volume', 'spread', 'real_volume', 'Date'], dtype='object')
Length of columns after resetting index: 8


ValueError: Length mismatch: Expected axis has 8 elements, new values have 6 elements

In [ ]:
# =============================================================================
# Automated trading script I - MACD
# Author : Mayank Rasu

# Please report bug/issues in the Q&A section
# =============================================================================


import MetaTrader5 as mt5
import os
import numpy as np
import pandas as pd
import datetime as dt
from stocktrends import Renko
import time
import copy

os.chdir(r"C:\DS Learning\Algorithmic Trading\Automated_Trading\Quant\MT5 Trading Terminal") #path where login credentials and server details
key = open("keys.txt","r").read().split()
path = r"C:\Program Files\MetaTrader 5\terminal64.exe"


# establish MetaTrader 5 connection to a specified trading account
if mt5.initialize(path=path,login=int(key[0]), password=key[1], server=key[2]):
    print("connection established")

#defining strategy parameters
pairs = ['EURUSD','GBPUSD','USDCHF','AUDUSD','USDCAD'] #currency pairs to be included in the strategy
pos_size = 0.5 #max capital allocated/position size for any currency pair. in MT5 the size is in unit of 10^5


def MACD(DF,a,b,c):
    """function to calculate MACD
       typical values a = 12; b =26, c =9"""
    df = DF.copy()
    df["MA_Fast"]=df["Close"].ewm(span=a,min_periods=a).mean()
    df["MA_Slow"]=df["Close"].ewm(span=b,min_periods=b).mean()
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"]
    df["Signal"]=df["MACD"].ewm(span=c,min_periods=c).mean()
    df.dropna(inplace=True)
    return (df["MACD"],df["Signal"])

def ATR(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2


def renko_DF(DF):
    "function to convert ohlc data into renko bricks"
    df = DF.copy()
    df.reset_index(inplace=True)
    df = df.iloc[:,[0,1,2,3,4,5]]
    df.columns = ["date","open","close","high","low","volume"]
    df2 = Renko(df)
    df2.brick_size = round(ATR(DF,120)["ATR"].iloc[-1],4)
    renko_df = df2.get_ohlc_data()
    renko_df["bar_num"] = np.where(renko_df["uptrend"]==True,1,np.where(renko_df["uptrend"]==False,-1,0))
    for i in range(1,len(renko_df["bar_num"])):
        if renko_df["bar_num"].iloc[i]>0 and renko_df["bar_num"].iloc[i-1]>0:
            renko_df["bar_num"].iloc[i]+=renko_df["bar_num"].iloc[i-1]
        elif renko_df["bar_num"].iloc[i]<0 and renko_df["bar_num"].iloc[i-1]<0:
            renko_df["bar_num"].iloc[i]+=renko_df["bar_num"].iloc[i-1]
    renko_df.drop_duplicates(subset="date",keep="last",inplace=True)
    return renko_df

def renko_merge(DF):
    "function to merging renko df with original ohlc df"
    df = copy.deepcopy(DF)
    df["Date"] = df.index
    renko = renko_DF(df)
    renko.columns = ["Date","open","high","low","close","uptrend","bar_num"]
    merged_df = df.merge(renko.loc[:,["Date","bar_num"]],how="outer",on="Date")
    merged_df["bar_num"].fillna(method='ffill',inplace=True)
    merged_df["macd"]= MACD(merged_df,12,26,9)[0]
    merged_df["macd_sig"]= MACD(merged_df,12,26,9)[1]
    return merged_df

def get_position_df():
    positions = mt5.positions_get()
    if len(positions) > 0:
        pos_df = pd.DataFrame(list(positions),columns=positions[0]._asdict().keys())
        pos_df.time = pd.to_datetime(pos_df.time, unit="s")
        pos_df.drop(['time_update', 'time_msc', 'time_update_msc', 'external_id'], axis=1, inplace=True)
        pos_df.type = np.where(pos_df.type==0,1,-1) #to distinguish between long and short positions
    else:
        pos_df = pd.DataFrame()
        
    return pos_df

def get_5m_candles(currency,lookback=10,bars=250):
    data = mt5.copy_rates_from(currency, mt5.TIMEFRAME_M5, dt.datetime.now() - dt.timedelta(10), 250)   
    data_df = pd.DataFrame(data) 
    data_df.time = pd.to_datetime(data_df.time, unit="s")
    data_df.set_index("time", inplace=True)
    data_df.rename(columns={"open":"Open","high":"High","low":"Low","close":"Close","volume":"Volume"},inplace=True)
    return data_df

def place_market_order(symbol,vol,buy_sell):
    if buy_sell.capitalize()[0] == "B":
        direction = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    else:
        direction = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": vol,
        "type": direction,
        "price": price,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    
    result = mt5.order_send(request)
    return result

def trade_signal(MERGED_DF,l_s):
    "function to generate signal"
    signal = ""
    df = copy.deepcopy(MERGED_DF)
    if l_s == "":
        if df["bar_num"].tolist()[-1]>=2 and df["macd"].tolist()[-1]>df["macd_sig"].tolist()[-1]:
            signal = "Buy"
        elif df["bar_num"].tolist()[-1]<=-2 and df["macd"].tolist()[-1]<df["macd_sig"].tolist()[-1]:
            signal = "Sell"
            
    elif l_s == "long":
        if df["bar_num"].tolist()[-1]<=-2 and df["macd"].tolist()[-1]<df["macd_sig"].tolist()[-1]:
            signal = "Close_Sell"
        elif df["macd"].tolist()[-1]<df["macd_sig"].tolist()[-1] and df["macd"].tolist()[-2]>df["macd_sig"].tolist()[-2]:
            signal = "Close"
            
    elif l_s == "short":
        if df["bar_num"].tolist()[-1]>=2 and df["macd"].tolist()[-1]>df["macd_sig"].tolist()[-1]:
            signal = "Close_Buy"
        elif df["macd"].tolist()[-1]>df["macd_sig"].tolist()[-1] and df["macd"].tolist()[-2]<df["macd_sig"].tolist()[-2]:
            signal = "Close"
    return signal
    

def main():
    try:
        open_pos = get_position_df()
        for currency in pairs:
            print(currency)
            long_short = ""
            if len(open_pos)>0:
                open_pos_cur = open_pos[open_pos["symbol"]==currency]
                if len(open_pos_cur)>0:
                    if (open_pos_cur.type * open_pos_cur.volume).sum() > 0:
                        long_short = "long"
                    elif (open_pos_cur.type * open_pos_cur.volume).sum() < 0:
                        long_short = "short"   
            
            ohlc = get_5m_candles(currency) 
            signal = trade_signal(renko_merge(ohlc),long_short)
    
            if signal == "Buy" or signal =="Sell":
                place_market_order(currency,pos_size,signal)
                print("New {} position initiated for {}".format(signal, currency))

            elif signal == "Close":
                tot_pos = (open_pos_cur.type * open_pos_cur.volume).sum()
                if tot_pos > 0:
                    place_market_order(currency,tot_pos,"Sell")
                elif tot_pos < 0:
                    place_market_order(currency,abs(tot_pos),"Buy")
                print("All positions closed for ", currency)
            elif signal == "Close_Buy":
                tot_pos = (open_pos_cur.type * open_pos_cur.volume).sum()
                place_market_order(currency,abs(tot_pos)+pos_size,"Buy")
                print("Existing Short position closed for ", currency)
                print("New long position initiated for ", currency)
            elif signal == "Close_Sell":
                tot_pos = (open_pos_cur.type * open_pos_cur.volume).sum()
                place_market_order(currency,tot_pos+pos_size,"Sell")
                print("Existing Long position closed for ", currency)
                print("New Short position initiated for ", currency)
    except Exception as e:
        print(e)
        print("error encountered....skipping this iteration")


# Continuous execution        
starttime=time.time()
timeout = time.time() + 60*60*1  # 60 seconds times 60 meaning the script will run for 1 hr
while time.time() <= timeout:
    try:
        print("passthrough at ",time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
        main()
        time.sleep(300 - ((time.time() - starttime) % 300.0)) # 5 minute interval between each new execution
    except KeyboardInterrupt:
        print('\n\nKeyboard exception received. Exiting.')
        exit()




connection established
passthrough at  2024-06-05 21:49:06
EURUSD
You are trying to merge on datetime64[ns] and object columns for key 'Date'. If you wish to proceed you should use pd.concat
error encountered....skipping this iteration


C:\Users\hugar\AppData\Local\Temp\ipykernel_21852\1371171013.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renko_df["bar_num"].iloc[i]+=renko_df["bar_num"].iloc[i-1]
C:\Users\hugar\AppData\Local\Temp\ipykernel_21852\1371171013.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renko_df["bar_num"].iloc[i]+=renko_df["bar_num"].iloc[i-1]
C:\Users\hugar\AppData\Local\Temp\ipykernel_21852\1371171013.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-



Keyboard exception received. Exiting.
passthrough at  2024-06-05 21:54:06
object of type 'NoneType' has no len()
error encountered....skipping this iteration
passthrough at  2024-06-05 21:59:06
object of type 'NoneType' has no len()
error encountered....skipping this iteration
